In [ ]:
from src.model import Config,GPT
from src.inference import GPTInfer,HNSWRetriever
import tiktoken
import torch
from sentence_transformers import SentenceTransformer


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
retriever = HNSWRetriever()

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'    # for apple macbook GPUs
print(f'using device: {device}')

In [ ]:
model_path = './model_00999.pt'
checkpoint = torch.load(model_path, weights_only=False)
model = GPT(config=checkpoint['config'])
model.load_state_dict(checkpoint['model'])
model = model.to(device)
token_encoder = tiktoken.get_encoding('gpt2')
generator = GPTInfer(model, token_encoder, device,retriever,embed_model)

In [ ]:
prompt = "Enter prompt here"

In [ ]:
tokens = generator.token_encoder.encode(prompt)
prev_tok = len(tokens)

In [ ]:
prompt = generator.print_stream(prompt, 1024,seed=42,temperature=0.7,end_prob=0.0)
tokens = generator.token_encoder.encode(prompt)
prev_tok = len(tokens)